# Business Performance & Customer Insights Analysis

**PROJECT:** Data Cleaning & Validation Phase
**Tools:** Python (Pandas, NumPy)

## Step 1: Import Libraries

In [11]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

warnings.filterwarnings('ignore')

print("=" * 80)
print("BUSINESS PERFORMANCE & CUSTOMER INSIGHTS ANALYSIS")
print("DATA CLEANING & VALIDATION PHASE")
print("=" * 80)
print("\nLibraries loaded successfully!\n")

BUSINESS PERFORMANCE & CUSTOMER INSIGHTS ANALYSIS
DATA CLEANING & VALIDATION PHASE

Libraries loaded successfully!



## Step 2: Load All Datasets

In [12]:
print("=" * 80)
print("STEP 1: Loading Datasets")
print("-" * 80)

df_sales = pd.read_csv('Sales_Transactions.csv')
print(f"Sales Transactions loaded: {len(df_sales):,} records")

df_customers = pd.read_csv('Customer_Master.csv')
print(f"Customer Master loaded: {len(df_customers):,} records")

df_products = pd.read_csv('Product_Master.csv')
print(f"Product Master loaded: {len(df_products):,} records")

df_date = pd.read_csv('Date_Dimension.csv')
print(f"Date Dimension loaded: {len(df_date):,} records")

print(f"\nTotal datasets loaded: 4")

STEP 1: Loading Datasets
--------------------------------------------------------------------------------
Sales Transactions loaded: 12,550 records
Customer Master loaded: 2,500 records
Product Master loaded: 400 records
Date Dimension loaded: 182 records

Total datasets loaded: 4


## Step 3: Initial Data Inspection

In [13]:
print("\n" + "=" * 80)
print("STEP 2: Initial Data Inspection")
print("-" * 80)

print("\n1. SALES TRANSACTIONS STRUCTURE:")
print(df_sales.info())

print("\n2. CUSTOMER MASTER STRUCTURE:")
print(df_customers.info())

print("\n3. PRODUCT MASTER STRUCTURE:")
print(df_products.info())

print("\n4. DATE DIMENSION STRUCTURE:")
print(df_date.info())


STEP 2: Initial Data Inspection
--------------------------------------------------------------------------------

1. SALES TRANSACTIONS STRUCTURE:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12550 entries, 0 to 12549
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   transaction_id     12550 non-null  int64  
 1   transaction_date   12550 non-null  object 
 2   customer_id        12550 non-null  int64  
 3   product_id         12550 non-null  object 
 4   category           12550 non-null  object 
 5   quantity           12550 non-null  int64  
 6   unit_price         12550 non-null  float64
 7   total_amount       12550 non-null  float64
 8   discount_applied   12550 non-null  float64
 9   payment_method     12450 non-null  object 
 10  store_location     12550 non-null  object 
 11  salesperson_id     12550 non-null  object 
 12  profit             12550 non-null  float64
 13  profit_margin_pct 

## Step 4: Data Quality Assessment

In [14]:
print("\n" + "=" * 80)
print("STEP 3: Data Quality Assessment")
print("-" * 80)

print("\nMISSING VALUES ANALYSIS:")
print("\nSales Transactions:")
sales_missing = df_sales.isnull().sum()
print(sales_missing[sales_missing > 0] if sales_missing.sum() > 0 else "No missing values")

print("\nCustomer Master:")
customer_missing = df_customers.isnull().sum()
print(customer_missing[customer_missing > 0] if customer_missing.sum() > 0 else "No missing values")

print("\nProduct Master:")
product_missing = df_products.isnull().sum()
print(product_missing[product_missing > 0] if product_missing.sum() > 0 else "No missing values")

print("\nDUPLICATE RECORDS ANALYSIS:")
sales_duplicates = df_sales.duplicated().sum()
customer_duplicates = df_customers.duplicated(subset=['customer_id']).sum()
product_duplicates = df_products.duplicated(subset=['product_id']).sum()

print(f"\nSales Transactions: {sales_duplicates} duplicate records found")
print(f"Customer Master: {customer_duplicates} duplicate customer IDs")
print(f"Product Master: {product_duplicates} duplicate product IDs")

print("\nDATA TYPE VALIDATION:")
print("\nSales - Date Column Type:", df_sales['transaction_date'].dtype)
print("Customer - Registration Date Type:", df_customers['registration_date'].dtype)
print("Date Dimension - Date Type:", df_date['date'].dtype)


STEP 3: Data Quality Assessment
--------------------------------------------------------------------------------

MISSING VALUES ANALYSIS:

Sales Transactions:
payment_method    100
dtype: int64

Customer Master:
email    125
phone     75
dtype: int64

Product Master:
No missing values

DUPLICATE RECORDS ANALYSIS:

Sales Transactions: 49 duplicate records found
Customer Master: 0 duplicate customer IDs
Product Master: 0 duplicate product IDs

DATA TYPE VALIDATION:

Sales - Date Column Type: object
Customer - Registration Date Type: object
Date Dimension - Date Type: object


## Step 5: Clean Sales Transactions

In [15]:
print("\n" + "=" * 80)
print("STEP 4: Cleaning Sales Transactions")
print("-" * 80)

original_sales_count = len(df_sales)

print("\n1. Removing duplicate transactions...")
df_sales_clean = df_sales.drop_duplicates()
duplicates_removed = original_sales_count - len(df_sales_clean)
print(f" - Removed {duplicates_removed} duplicate records")
print(f" - Clean records: {len(df_sales_clean):,}")

print("\n2. Converting transaction_date to datetime...")
df_sales_clean['transaction_date'] = pd.to_datetime(df_sales_clean['transaction_date'])
print(" - Date column converted successfully")
print(f" - Date range: {df_sales_clean['transaction_date'].min()} to {df_sales_clean['transaction_date'].max()}")

print("\n3. Handling missing payment methods...")
missing_payment = df_sales_clean['payment_method'].isnull().sum()
print(f" - Found {missing_payment} records with missing payment method")

df_sales_clean['payment_method'].fillna('Unknown', inplace=True)
print(" - Filled missing values with 'Unknown'")

print("\n4. Validating quantity column...")
if df_sales_clean['quantity'].dtype == 'object':
    print(" - Warning: Quantity column has mixed data types")
    df_sales_clean['quantity'] = pd.to_numeric(df_sales_clean['quantity'], errors='coerce')
    qty_nulls = df_sales_clean['quantity'].isnull().sum()
    if qty_nulls > 0:
        print(f" - Converted {qty_nulls} invalid quantities to null")
        df_sales_clean['quantity'].fillna(df_sales_clean['quantity'].median(), inplace=True)
        print(" - Filled nulls with median quantity")
else:
    print(" - Quantity column is already numeric")

print("\n5. Detecting and handling outliers in transaction amounts...")
Q1 = df_sales_clean['total_amount'].quantile(0.25)
Q3 = df_sales_clean['total_amount'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 3 * IQR
upper_bound = Q3 + 3 * IQR

outliers = df_sales_clean[(df_sales_clean['total_amount'] < lower_bound) |
                          (df_sales_clean['total_amount'] > upper_bound)]
print(f" - Found {len(outliers)} outlier transactions")
print(f" - Outlier range: INR {outliers['total_amount'].min():,.2f} to INR {outliers['total_amount'].max():,.2f}")

df_sales_clean['is_outlier'] = ((df_sales_clean['total_amount'] < lower_bound) |
                                 (df_sales_clean['total_amount'] > upper_bound)).astype(int)
print(" - Outliers flagged for review (not removed)")

print("\n6. Validating discount percentages...")
invalid_discounts = df_sales_clean[(df_sales_clean['discount_applied'] < 0) |
                                    (df_sales_clean['discount_applied'] > 100)]
print(f" - Invalid discounts: {len(invalid_discounts)}")
if len(invalid_discounts) > 0:
    df_sales_clean.loc[invalid_discounts.index, 'discount_applied'] = 0
    print(" - Reset invalid discounts to 0")


STEP 4: Cleaning Sales Transactions
--------------------------------------------------------------------------------

1. Removing duplicate transactions...
 - Removed 49 duplicate records
 - Clean records: 12,501

2. Converting transaction_date to datetime...
 - Date column converted successfully
 - Date range: 2024-01-01 00:00:00 to 2024-06-30 00:00:00

3. Handling missing payment methods...
 - Found 100 records with missing payment method
 - Filled missing values with 'Unknown'

4. Validating quantity column...
 - Quantity column is already numeric

5. Detecting and handling outliers in transaction amounts...
 - Found 1608 outlier transactions
 - Outlier range: INR 79,068.90 to INR 1,623,312.00
 - Outliers flagged for review (not removed)

6. Validating discount percentages...
 - Invalid discounts: 0


## Step 6: Clean Customer Master

In [16]:
print("\n" + "=" * 80)
print("STEP 5: Cleaning Customer Master")
print("-" * 80)

original_customer_count = len(df_customers)

print("\n1. Checking for duplicate customer IDs...")
df_customers_clean = df_customers.drop_duplicates(subset=['customer_id'])
cust_dup_removed = original_customer_count - len(df_customers_clean)
print(f" - Removed {cust_dup_removed} duplicate customer records")

print("\n2. Standardizing city names...")
city_fixes = {
    'mumbai': 'Mumbai',
    'Mumbai ': 'Mumbai',
    'Mmbai': 'Mumbai',
    'delhi': 'Delhi',
    'bangalore': 'Bangalore'
}

typos_fixed = 0
for typo, correct in city_fixes.items():
    typo_count = (df_customers_clean['city'] == typo).sum()
    df_customers_clean['city'] = df_customers_clean['city'].replace(typo, correct)
    typos_fixed += typo_count

print(f" - Fixed {typos_fixed} city name typos")
print(f" - Unique cities: {df_customers_clean['city'].nunique()}")

print("\n3. Handling missing emails...")
missing_emails = df_customers_clean['email'].isnull().sum()
print(f" - Found {missing_emails} customers without email")
df_customers_clean['email'].fillna('email_not_provided@unknown.com', inplace=True)
print(" - Filled missing emails with placeholder")

print("\n4. Handling missing phone numbers...")
missing_phones = df_customers_clean['phone'].isnull().sum()
print(f" - Found {missing_phones} customers without phone")
df_customers_clean['phone'].fillna('Phone Not Available', inplace=True)
print(" - Filled missing phones with placeholder")

print("\n5. Converting registration_date to datetime...")
df_customers_clean['registration_date'] = pd.to_datetime(df_customers_clean['registration_date'])
print(" - Date column converted successfully")

print("\n6. Validating age groups...")
def validate_age_group(row):
    age = row['age']
    if age < 26: return "18-25"
    elif age < 36: return "26-35"
    elif age < 46: return "36-45"
    elif age < 61: return "46-60"
    else: return "60+"

df_customers_clean['correct_age_group'] = df_customers_clean.apply(validate_age_group, axis=1)
mismatches = (df_customers_clean['age_group'] != df_customers_clean['correct_age_group']).sum()
print(f" - Found {mismatches} age group mismatches")
if mismatches > 0:
    df_customers_clean['age_group'] = df_customers_clean['correct_age_group']
    print(" - Corrected age groups")
df_customers_clean.drop('correct_age_group', axis=1, inplace=True)


STEP 5: Cleaning Customer Master
--------------------------------------------------------------------------------

1. Checking for duplicate customer IDs...
 - Removed 0 duplicate customer records

2. Standardizing city names...
 - Fixed 32 city name typos
 - Unique cities: 8

3. Handling missing emails...
 - Found 125 customers without email
 - Filled missing emails with placeholder

4. Handling missing phone numbers...
 - Found 75 customers without phone
 - Filled missing phones with placeholder

5. Converting registration_date to datetime...
 - Date column converted successfully

6. Validating age groups...
 - Found 0 age group mismatches


## Step 7: Clean Product Master

In [17]:
print("\n" + "=" * 80)
print("STEP 6: Cleaning Product Master")
print("-" * 80)

print("\n1. Checking for duplicate products...")
df_products_clean = df_products.drop_duplicates(subset=['product_id'])
prod_dup_removed = len(df_products) - len(df_products_clean)
print(f" - Removed {prod_dup_removed} duplicate product records")

print("\n2. Validating pricing logic (cost_price <= selling_price)...")
invalid_pricing = df_products_clean[df_products_clean['cost_price'] > df_products_clean['selling_price']]
print(f" - Found {len(invalid_pricing)} products with cost > selling price")
if len(invalid_pricing) > 0:
    df_products_clean.loc[invalid_pricing.index, ['cost_price', 'selling_price']] = \
        df_products_clean.loc[invalid_pricing.index, ['selling_price', 'cost_price']].values
    print(f" - Corrected pricing for {len(invalid_pricing)} products")

print("\n3. Validating stock quantities...")
negative_stock = df_products_clean[df_products_clean['stock_quantity'] < 0]
print(f" - Found {len(negative_stock)} products with negative stock")
if len(negative_stock) > 0:
    df_products_clean.loc[negative_stock.index, 'stock_quantity'] = 0
    print(" - Reset negative stock to 0")


STEP 6: Cleaning Product Master
--------------------------------------------------------------------------------

1. Checking for duplicate products...
 - Removed 0 duplicate product records

2. Validating pricing logic (cost_price <= selling_price)...
 - Found 0 products with cost > selling price

3. Validating stock quantities...
 - Found 0 products with negative stock


## Step 8: Clean Date Dimension

In [18]:
print("\n" + "=" * 80)
print("STEP 7: Cleaning Date Dimension")
print("-" * 80)

print("\n1. Converting date column to datetime...")
df_date_clean = df_date.copy()
df_date_clean['date'] = pd.to_datetime(df_date_clean['date'])
print(" - Date column converted successfully")


STEP 7: Cleaning Date Dimension
--------------------------------------------------------------------------------

1. Converting date column to datetime...
 - Date column converted successfully


## Step 9: Cross-Table Validation

In [19]:
print("\n" + "=" * 80)
print("STEP 8: Cross-Table Validation (Data Integrity)")
print("-" * 80)

print("\n1. Checking customer_id integrity...")
valid_customer_ids = set(df_customers_clean['customer_id'])
sales_customer_ids = set(df_sales_clean['customer_id'])
orphan_customers = sales_customer_ids - valid_customer_ids
print(f" - Found {len(orphan_customers)} orphan customer IDs in sales")
if len(orphan_customers) > 0:
    print(f" - Warning: {len(orphan_customers)} transactions reference non-existent customers")

print("\n2. Checking product_id integrity...")
valid_product_ids = set(df_products_clean['product_id'])
sales_product_ids = set(df_sales_clean['product_id'])
orphan_products = sales_product_ids - valid_product_ids
print(f" - Found {len(orphan_products)} orphan product IDs in sales")
if len(orphan_products) > 0:
    print(f" - Warning: {len(orphan_products)} transactions reference non-existent products")

print("\n3. Checking transaction date coverage...")
min_txn_date = df_sales_clean['transaction_date'].min()
max_txn_date = df_sales_clean['transaction_date'].max()
min_dim_date = df_date_clean['date'].min()
max_dim_date = df_date_clean['date'].max()

print(f" - Transaction date range: {min_txn_date.date()} to {max_txn_date.date()}")
print(f" - Date dimension range: {min_dim_date.date()} to {max_dim_date.date()}")

if min_txn_date >= min_dim_date and max_txn_date <= max_dim_date:
    print(" - All transaction dates covered by date dimension")
else:
    print(" - Warning: Some transaction dates outside date dimension range")


STEP 8: Cross-Table Validation (Data Integrity)
--------------------------------------------------------------------------------

1. Checking customer_id integrity...
 - Found 0 orphan customer IDs in sales

2. Checking product_id integrity...
 - Found 0 orphan product IDs in sales

3. Checking transaction date coverage...
 - Transaction date range: 2024-01-01 to 2024-06-30
 - Date dimension range: 2024-01-01 to 2024-06-30
 - All transaction dates covered by date dimension


## Step 10: Final Data Quality Summary & Export

In [20]:
print("\n" + "=" * 80)
print("DATA CLEANING SUMMARY")
print("=" * 80)

print("\nBEFORE vs AFTER CLEANING:")
print(f"\nSales Transactions:")
print(f" - Original records: {original_sales_count:,}")
print(f" - Clean records: {len(df_sales_clean):,}")
print(f" - Removed: {original_sales_count - len(df_sales_clean):,}")

print(f"\nCustomer Master:")
print(f" - Original records: {original_customer_count:,}")
print(f" - Clean records: {len(df_customers_clean):,}")
print(f" - Removed: {original_customer_count - len(df_customers_clean):,}")

print(f"\nProduct Master:")
print(f" - Original records: {len(df_products):,}")
print(f" - Clean records: {len(df_products_clean):,}")
print(f" - Removed: {len(df_products) - len(df_products_clean):,}")

print("\nDATA QUALITY IMPROVEMENTS:")
print(" - Duplicates removed from all tables")
print(" - Missing values handled appropriately")
print(" - Date columns converted to datetime")
print(" - City names standardized")
print(" - Pricing logic validated")
print(" - Outliers flagged for analysis")
print(" - Cross-table integrity checked")

print("\n" + "=" * 80)
print("STEP 9: Saving Cleaned Data")
print("-" * 80)

df_sales_clean.to_csv('Sales_Transactions_Clean.csv', index=False)
print("Saved Sales_Transactions_Clean.csv")

df_customers_clean.to_csv('Customer_Master_Clean.csv', index=False)
print("Saved Customer_Master_Clean.csv")

df_products_clean.to_csv('Product_Master_Clean.csv', index=False)
print("Saved Product_Master_Clean.csv")

df_date_clean.to_csv('Date_Dimension_Clean.csv', index=False)
print("Saved Date_Dimension_Clean.csv")

print("\n" + "=" * 80)
print("DATA CLEANING COMPLETED SUCCESSFULLY")
print("=" * 80)
print("\nCleaned datasets ready for Exploratory Data Analysis (EDA)")
print("- All data quality issues addressed")
print("- Data consistency improved")
print("- Reporting accuracy enhanced")


DATA CLEANING SUMMARY

BEFORE vs AFTER CLEANING:

Sales Transactions:
 - Original records: 12,550
 - Clean records: 12,501
 - Removed: 49

Customer Master:
 - Original records: 2,500
 - Clean records: 2,500
 - Removed: 0

Product Master:
 - Original records: 400
 - Clean records: 400
 - Removed: 0

DATA QUALITY IMPROVEMENTS:
 - Duplicates removed from all tables
 - Missing values handled appropriately
 - Date columns converted to datetime
 - City names standardized
 - Pricing logic validated
 - Outliers flagged for analysis
 - Cross-table integrity checked

STEP 9: Saving Cleaned Data
--------------------------------------------------------------------------------
Saved Sales_Transactions_Clean.csv
Saved Customer_Master_Clean.csv
Saved Product_Master_Clean.csv
Saved Date_Dimension_Clean.csv

DATA CLEANING COMPLETED SUCCESSFULLY

Cleaned datasets ready for Exploratory Data Analysis (EDA)
- All data quality issues addressed
- Data consistency improved
- Reporting accuracy enhanced
